In [ ]:
# crash colab to get more RAM
# !kill -9 -1

In [ ]:
!nvidia-smi

Thu Aug  3 16:16:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/TIU-conf

/content/drive/MyDrive/TIU-conf


In [ ]:
!pip install -U accelerate
! pip install -U transformers
!pip install accelerate -U


In [ ]:
!pip install datasets
!pip install rouge_score
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=18f85e3e669392e4f8dee49340d3665e93ff50dfb11bd55834299a588d8b5341
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
train_dataset_A = load_dataset("scientific_papers", "pubmed", split="train")
val_dataset_A = load_dataset("scientific_papers", "pubmed", split="validation")
test_dataset_A = load_dataset("scientific_papers", "pubmed", split="test")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [ ]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["abstract"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
train_dataset1 = train_dataset_A.select(range(2000))
val_dataset1 = val_dataset_A.select(range(250))
test_dataset1 = test_dataset_A.select(range(250))

In [ ]:
test_dataset1

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 250
})

In [ ]:
train_dataset = train_dataset1.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
val_dataset = val_dataset1.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [ ]:
# set generate hyperparameters
led.config.num_beams = 4
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [ ]:
rouge = load_metric("rouge")

<ipython-input-20-6f1857dc142f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="/content/drive/MyDrive/TIU-con/LED",
    logging_steps=5,
    eval_steps=10,
    save_steps=200,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
)

In [ ]:
import os
#os._exit(00)

In [ ]:
import accelerate

accelerate.__version__

'0.21.0'

In [ ]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
trainer = Seq2SeqTrainer(
    model=led,
    #model = LEDForConditionalGeneration.from_pretrained("/content/drive/MyDrive/TIU-con/TR/checkpoint-680").to("cuda").half(),
    #model=AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/TIU-con/TR/checkpoint-680", gradient_checkpointing=True, use_cache=False),
    tokenizer=tokenizer,
    args=training_args,
    #compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
#download a folder from google drive to local drive
#from google.colab import files
#files.download("/content/drive/MyDrive/TIU-con")

In [ ]:
#Finetuning of LED model using PubMed dataset(2000,250,250)
#Trained_LED=trainer.train()

In [ ]:
#Explicitely you can save the finetuned model to your drive
#trainer.save_model("/content/drive/MyDrive/TIU-con/Trainer")

## Evaluation

In [ ]:
import torch

from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

# load pubmed
#pubmed_test = load_dataset("scientific_papers", "pubmed", ignore_verifications=True, split="test")

# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("/content/drive/MyDrive/TIU-con/LED/checkpoint-2400")
model = LEDForConditionalGeneration.from_pretrained("/content/drive/MyDrive/TIU-con/LED/checkpoint-2400").to("cuda").half()
#model=Trained_LED

In [ ]:
test_dataset = test_dataset1.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "section_names"],
)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
# function to make inference on a batch of dataset

def generate_answer(batch):
  inputs_dict = tokenizer(batch["article"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch

In [ ]:
# making inference on the without processed test dataset and generating summaries
result1 = test_dataset1.map(generate_answer, batched=True, batch_size=4)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# save to drive
import csv
result1.to_csv("/content/drive/MyDrive/TIU-con/LED/LED_result.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

4911011

In [ ]:
#Checking predicted results csv files total rows.
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED/LED_result.csv")
data

,article,abstract,section_names,predicted_abstract
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in pa...,1. Introduction\n2. Methods\n3. Results\n4. Di...,abstractanxiety affects quality of life in th...
1,small non - coding rnas are transcribed into m...,"small non - coding rnas include sirna , mirna...",Introduction\nAberrant Expression of miRNA in ...,breast cancer is caused by multiple processes...
2,ohss is a serious complication of ovulation in...,objective : to evaluate the efficacy and safe...,Introduction\nMaterials and Methods\nResults\n...,purposeto evaluate the efficacy and safety of...
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of ...,I\nM\nR\nD,backgroundcongenital adrenal hyperplasia ( ca...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodul...,Introduction\nMaterials and Methods\nDrug and ...,type 1 diabetes ( t1d ) results from the dest...
...,...,...,...,...
245,renal clear cell carcinomas represent about 3%...,renal clear cell carcinomas represent about 3...,Introduction\nPatients and methods\nStatistica...,backgroundrenal clear cell carcinomas represe...
246,intracranial hypotension syndrome ( ihs ) is c...,intracranial hypotension syndrome typically o...,INTRODUCTION\nCASE REPORT\nDISCUSSION\nCONCLUSION,abstractintracranial hypotension syndrome ( i...
247,pituitary insufficiency in childhood is often ...,we report the cases of two japanese childre...,Introduction\nCase Reports\nCase 1\nCase 2\nDi...,xanthogranuloma of the sellar is a rare pitui...
248,a 52-year - old female patient visited the hos...,breast surgery is a common procedure performe...,Case Report\nDiscussion,a 52-year - old female patient visited the ho...


In [ ]:
# making inference on the processed test dataset and generating summaries
result = test_dataset.map(generate_answer, batched=True, batch_size=4)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# save to drive
import csv
result.to_csv("/content/drive/MyDrive/TIU-con/LED/LED_result_Processed.csv")
import pandas as pd
datap = pd.read_csv("/content/drive/MyDrive/TIU-con/LED/LED_result_Processed.csv")
datap

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

,article,abstract,input_ids,attention_mask,global_attention_mask,labels,predicted_abstract
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in pa...,[ 0 260 43537 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 557 15 5 8819 9 6882 ...,abstractanxiety affects quality of life in th...
1,small non - coding rnas are transcribed into m...,"small non - coding rnas include sirna , mirna...",[ 0 23115 786 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 650 786 111 25776 910 22118 6...,breast cancer is caused by multiple processes...
2,ohss is a serious complication of ovulation in...,objective : to evaluate the efficacy and safe...,[ 0 2678 7485 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 4554 4832 7 10516 5 22081 ...,purposeto evaluate the efficacy and safety of...
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of ...,[ 0 37519 225 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 36764 8632 42761 337 8944 2911 190...,backgroundcongenital adrenal hyperplasia ( ca...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodul...,[ 0 12528 112 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 4554 1640 29 3256 33475 4325 45...,type 1 diabetes ( t1d ) results from the dest...
...,...,...,...,...,...,...,...
245,renal clear cell carcinomas represent about 3%...,renal clear cell carcinomas represent about 3...,[ 0 2558 337 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 39729 699 3551 30708 27745 3594 ...,backgroundrenal clear cell carcinomas represe...
246,intracranial hypotension syndrome ( ihs ) is c...,intracranial hypotension syndrome typically o...,[ 0 2544 13249 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 19567 1043 3917 2617 43851 17699 141...,abstractintracranial hypotension syndrome ( i...
247,pituitary insufficiency in childhood is often ...,we report the cases of two japanese childre...,[ 0 17291 6439 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 1437 1437 52 266 5 1200 ...,xanthogranuloma of the sellar is a rare pitui...
248,a 52-year - old female patient visited the hos...,breast surgery is a common procedure performe...,[ 0 102 3135 ... 1 1 1],[1 1 1 ... 0 0 0],[1 0 0 ... 0 0 0],[ 0 6181 3012 16 10 1537 7089 37...,a 52-year - old female patient visited the ho...


In [ ]:
#Another way check that
output=len(result)
with open('/content/drive/MyDrive/TIU-con/LED/test.csv', mode='w') as csv_file:
  fieldnames = ['Article', 'Actual Abstract','Predictive Abstract']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()
  for i in range(output):
    writer.writerow({'Article':result['article'][i],'Actual Abstract':result['abstract'][i],'Predictive Abstract':result['predicted_abstract'][i]})
#checking rows in file
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED/test.csv")
data